In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

In [3]:
with open('./benchmark_data/Anaheim_flow.txt') as Anaheim_flow:
    Anaheim_flow_lines = Anaheim_flow.readlines()
Anaheim_links = []
i = -1
for line in Anaheim_flow_lines:
    i += 1
    if i > 0:
        Anaheim_links.append(line.split('\t')[1:3])
numLinks = i

In [4]:
link_list_js = [str(int(Anaheim_links[i][0])) + ',' + str(int(Anaheim_links[i][1])) for \
             i in range(len(Anaheim_links))]

link_list_pk = [str(int(Anaheim_links[i][0])) + '->' + str(int(Anaheim_links[i][1])) for \
             i in range(len(Anaheim_links))]

In [5]:
zdump(link_list_js, '../temp_files/Anaheim_link_list_js.pkz')

zdump(link_list_pk, '../temp_files/Anaheim_link_list_pk.pkz')

In [6]:
numNodes = max([int(Anaheim_links[i][1]) for i in range(numLinks)])

In [7]:
numNodes

416

In [8]:
from collections import defaultdict

node_neighbors_dict = defaultdict(list)

for node in range(numNodes):
    for link in Anaheim_links:
        if node == int(link[0]):
            node_neighbors_dict[str(node)].append(int(link[1]))

In [9]:
node_neighbors_dict['10']

[338, 362]

In [10]:
with open('./benchmark_data/Anaheim_trips.txt') as Anaheim_trips:
    Anaheim_trips_lines = Anaheim_trips.readlines()

In [11]:
numZones = int(Anaheim_trips_lines[0].split(' ')[3])

In [12]:
numZones

38

In [13]:
od_pairs = []
for i in range(numZones+1)[1:]:
    for j in range(numZones+1)[1:]:
        if i != j:
            od_pairs.append([i, j])

In [14]:
od_pairs;

In [15]:
# create O-D pair labels
# create a dictionary mapping O-D pairs to labels

import json

OD_pair_label_dict = {}
OD_pair_label_dict_ = {}

label = 1
for i in range(numZones + 1)[1:]:
    for j in range(numZones + 1)[1:]:
        key = (i, j)
        OD_pair_label_dict[str(key)] = label
        OD_pair_label_dict_[str(label)] = key
        label += 1
        
with open('../temp_files/od_pair_label_dict_Anaheim.json', 'w') as json_file:
    json.dump(OD_pair_label_dict, json_file)
    
with open('../temp_files/od_pair_label_dict__Anaheim.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_, json_file)


OD_pair_label_dict_refined = {}
OD_pair_label_dict_refined_ = {}

label = 1
for i in range(numZones + 1)[1:]:
    for j in range(numZones + 1)[1:]:
        if i != j:
            key = (i, j)
            OD_pair_label_dict_refined[str(key)] = label
            OD_pair_label_dict_refined_[str(label)] = key
            label += 1
        
with open('../temp_files/od_pair_label_dict_Anaheim_refined.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_refined, json_file)
    
with open('../temp_files/od_pair_label_dict__Anaheim_refined.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_refined_, json_file)
    
    
# create link labels
# create a dictionary mapping directed links to labels
link_label_dict = {}
link_label_dict_ = {}

link_list = zload('../temp_files/Anaheim_link_list_js.pkz')

for i in range(numLinks):
    link_label_dict[str(i)] = link_list[i]

for i in range(numLinks):
    link_label_dict_[link_list[i]] = i

with open('../temp_files/link_label_dict_Anaheim.json', 'w') as json_file:
    json.dump(link_label_dict, json_file)
    
with open('../temp_files/link_label_dict_Anaheim_.json', 'w') as json_file:
    json.dump(link_label_dict_, json_file)
    
# create link labels
# create a dictionary mapping directed links to labels
link_label_dict = {}
link_label_dict_ = {}

link_list = zload('../temp_files/Anaheim_link_list_pk.pkz')

for i in range(numLinks):
    link_label_dict[str(i)] = link_list[i]

for i in range(numLinks):
    link_label_dict_[link_list[i]] = i

zdump(link_label_dict, '../temp_files/link_label_dict_Anaheim_network.pkz')
zdump(link_label_dict_, '../temp_files/link_label_dict_Anaheim_network_.pkz')

In [16]:
link_label_dict_;

In [17]:
link_length_dict_Anaheim = {}  # save free-flow time actually
link_capac_dict_Anaheim = {}

length_list = []
capac_list = []
with open('./benchmark_data/Anaheim_net.txt', 'r') as f:
    read_data = f.readlines()
    flag = 0
    for row in read_data:
        if ';' in row:
            flag += 1
            if flag > 1:
                length_list.append(float(row.split('\t')[5]))
                capac_list.append(float(row.split('\t')[3]))
                
for idx in range(len(length_list)):
    key = str(idx)
    link_length_dict_Anaheim[key] = length_list[idx]
    link_capac_dict_Anaheim[key] = capac_list[idx]

with open('../temp_files/link_length_dict_Anaheim.json', 'w') as json_file:
    json.dump(link_length_dict_Anaheim, json_file)
    
with open('../temp_files/link_capac_dict_Anaheim.json', 'w') as json_file:
    json.dump(link_capac_dict_Anaheim, json_file)

In [18]:
link_label_dict = zload('../temp_files/link_label_dict_Anaheim_network.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_Anaheim_network_.pkz')

In [19]:
keyLink = str(link_label_dict_['1->117'])
link_length_dict_Anaheim[keyLink]

1.090458488

In [20]:
# compute length of a route
def routeLength(route):
    link_list = []
    node_list = []
    for i in route.split('->'):
        node_list.append(int(i))
    for i in range(len(node_list))[:-1]:
        link_list.append('%d->%d' %(node_list[i], node_list[i+1]))
    length_of_route = sum([link_length_dict_Anaheim[str(link_label_dict_[link])] for link in link_list])
    return length_of_route

In [21]:
routeLength('1->117')

1.090458488

In [22]:
node_neighbors_dict;

In [23]:
len(length_list)

914

In [24]:
link_list_js;

In [25]:
import networkx as nx

Anaheim = nx.DiGraph()

Anaheim.add_nodes_from(range(numNodes+1)[1:])

Anaheim_weighted_edges = [(int(link_list_js[i].split(',')[0]), int(link_list_js[i].split(',')[1]), \
                           length_list[i]) for i in range(len(link_list_js))]

Anaheim.add_weighted_edges_from(Anaheim_weighted_edges)

path = nx.all_pairs_dijkstra_path(Anaheim)

with open('../temp_files/path-link_incidence_Anaheim.txt', 'w') as the_file:
    for od in od_pairs:
        origi = od[0]
        desti = od[1]
        the_file.write('O-D pair (%s, %s):\n'%(origi, desti))
        route = str(path[origi][desti]).replace("[", "").replace(", ", "->").replace("]", "")
        the_file.write(route)
        the_file.write('\n')

In [26]:
with open('../temp_files/path-link_incidence_Anaheim.txt', 'r') as the_file:
    # path counts
    i = 0  
    for row in the_file:
        if '->' in row:
            i = i + 1

with open('../temp_files/numRoutes_Anaheim.json', 'w') as json_file:
    json.dump(i, json_file)